<a href="https://colab.research.google.com/github/bartoszkozakiewicz/PARAGON_AI_BACKEND/blob/main/ParagonAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install easyocr
!pip install transformers
!pip install datasets

#1 OCR CONTENT

In [22]:
import easyocr
import cv2
from google.colab.patches import cv2_imshow
#For the first time it downloads the models for the languages chosen below.
#Not all languages are compatible with each other so you cannot put
#multiple languages below
#reader = easyocr.Reader(['hi', 'te', 'en'])  #Hindi, telugu, and English
#The above gives error that Telugu is only compatible with English.

#So let us just use Hindi and English
#To use GPU you need to have CUDA configured for the pytorch library.
reader = easyocr.Reader(['pl'], gpu=True)  #Hindi, telugu, and English

img = cv2.imread('/content/1.png')


In [23]:
results = reader.readtext(img, detail=1, paragraph=False) #Set detail to 0 for simple text output , paragraph=False

SHOW IMAGE WITH BOUNDING BOXES

In [ ]:
for (bbox, text, prob) in results:

    #Define bounding boxes
    (tl, tr, br, bl) = bbox
    tl = (int(tl[0]), int(tl[1]))
    tr = (int(tr[0]), int(tr[1]))
    br = (int(br[0]), int(br[1]))
    bl = (int(bl[0]), int(bl[1]))

    #Remove non-ASCII characters to display clean text on the image (using opencv)
    text = "".join([c if ord(c) < 128 else "" for c in text]).strip()

    #Put rectangles and text on the image
    cv2.rectangle(img, tl, br, (0, 255, 0), 2)
    cv2.putText(img, text, (tl[0], tl[1] - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
# show the output image
cv2_imshow(img)
cv2.waitKey(0)

# 2 START OF DATA PREPARATION

In [6]:
import pandas as pd
import numpy as np

prepared_results = []

for idx,result in enumerate(results):
  bbox = result[0][0] + result[0][2]
  prepared_results.append({"x0":int(bbox[0]),"y0":int(bbox[1]),"x2":int(bbox[2]),"y2":int(bbox[3]) , "word": result[1]})

data_to_json = prepared_results
dataframe = pd.DataFrame(prepared_results, dtype=np.int16)
dataframe.head(5)

<ipython-input-6-77bac07d40da>:11: FutureWarning: Could not cast to int16, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised.
  dataframe = pd.DataFrame(prepared_results, dtype=np.int16)


,x0,y0,x2,y2,word
0,223,33,265,53,a
1,332,24,380,48,'
2,75,45,113,59,
3,114,39,223,60,6' !
4,111,59,157,75,4-


In [12]:
import json
# Write the data to a JSON file
with open('output.json', 'w') as json_file:
    json.dump(data_to_json, json_file, indent=4)

**PODZIAŁ**
- PRODUCT
- PRODUCT_PRICE
- AMOUNT

- SUM_PRICE
- COMPANY
- DATE
- O

- DISC
- DISC_PRICE

- ADDRESS


# 3 Model preparation

### 3.1 Prapare LayoutLM model configuration - CONFIG

In [18]:
#DOPRECYZOWAC
config = {
    "tokenizer_name": "microsoft/layoutlm-base-uncased",
    "model_name": "microsoft/layoutlm-base-uncased",
    "n_samples": None,  # None for full dataset
    "model_path": "model",
    "num_labels": 5,
    "pct_train": 0.8,
    "learning_rate": 0.001,
    "batch_size_train": 16,
    "batch_size_test": 1,
    "epochs": 100,
    "log_freq": 2,
}


### 3.2 Take tokenizers etc from hugginface - MODEL

In [ ]:
from transformers import (
    LayoutLMTokenizer,
    LayoutLMConfig,
    LayoutLMForTokenClassification,
    AdamW
)
import torch
from datasets import load_dataset, Features, Sequence, ClassLabel, Value, Array2D


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = LayoutLMTokenizer.from_pretrained(
    config["tokenizer_name"],
)
model = LayoutLMForTokenClassification.from_pretrained(
    config["model_name"],num_labels=config["num_labels"]
    )

model.to(device)

### 3.3 Preprocess data

#### 3.3.1 Normalize bbox

In [ ]:
from PIL import Image
img_path = "/content/1.jpg"
image = Image.open(name_of_your_document).convert("RGB")

width, height = image.size

def normalize_bbox(bbox, width, height):
    return [
        int(1000 * (bbox[0] / width)),
        int(1000 * (bbox[1] / height)),
        int(1000 * (bbox[2] / width)),
        int(1000 * (bbox[3] / height)),
    ]

In [25]:
data = {
    "words":["a",":","asd","jan"],
    "bboxes":[[89,2,5,7],[214,23,5,23],[213,32,43,43],[32,23,65,75]],
    "labels":[1,1,4,7]
}

#### 3.3.2 Prepare dataset for model to train

In [33]:
def final_data_preparation(data,max_length=512):
  final_bbox = []
  final_labels = []
  for word,box,label in zip(data["words"],data["bboxes"],data["labels"]):
    tokenized_word = tokenizer.tokenize(word)
    #Prepare final bbox
    final_bbox.extend([box]*len(tokenized_word))

    #Prepare final labels
    final_labels.append(label)
    final_labels.extend([-1 for _ in range(len(tokenized_word)-1)]) # If repeates then make -1 of that ?????????

  num_special_tokens = 2
  if len(final_bbox) > max_length - num_special_tokens:
    final_bbox = final_bbox[:(max_length-num_special_tokens)]
    final_labels = final_labels[:(max_length-num_special_tokens)]

  #Lastly make usage of special tokens in bbox and labels
  final_bbox =  [[0, 0, 0, 0]] + final_bbox + [[1000, 1000, 1000, 1000]]
  final_labels =  [0] + final_labels + [0]

  #Create encoding to get - input_ids, attention_mask, token_type_ids
  encoding = tokenizer(" ".join(data["words"]),padding="max_length",truncation=True)
  print("ENCODING",encoding)

  #If sentence was smaller than max_length then it have to be padding added to final_box and final_labels to keep compatibility
  pad_token_box = [0, 0, 0, 0]
  padding_length = max_length - len(tokenizer(' '.join(data["words"]), truncation=True)["input_ids"])
  final_bbox += [pad_token_box] * padding_length
  final_labels += [0] * padding_length


  ## Fill encoding with previously prepared bboxes and labels
  encoding['bboxes'] = final_bbox
  encoding['ner_tags'] = final_labels

  return encoding


data_result = final_data_preparation(data)

#Prepare training features for
# training_features = Features({
#     'input_ids': Sequence(Value(dtype='int64')),
#     'bboxes': Array2D(dtype="int64", shape=(512, 4)),
#     'attention_mask': Sequence(Value(dtype='int64')),
#     'token_type_ids': Sequence(Value(dtype='int64')),
#     'ner_tags': Sequence(ClassLabel(names=id2label)),
#     'image_path': Value(dtype='string'),
#     'words': Sequence(Value(dtype='string')),
# })



ENCODING {'input_ids': [101, 1037, 1024, 2004, 2094, 5553, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [38]:
len(data_result["input_ids"]),len(data_result["bboxes"]),len(data_result["attention_mask"]),len(data_result["token_type_ids"]),len(data_result["ner_tags"])

(512, 512, 512, 512, 512)

Get data into torch dataloaders for training purposes

In [45]:

from torch.utils.data import Dataset, DataLoader

class LayoutLMDataset(Dataset):
    def __init__(self, input_ids,bboxes, attention_mask, token_type_ids,labels):
        self.input_ids = input_ids
        self.bboxes=bboxes
        self.attention_mask = attention_mask
        self.token_type_ids = token_type_ids
        self.labels = labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        input_id = self.input_ids[idx]
        bbox = self.bboxes[idx]
        attention_mask = self.attention_mask[idx]
        token_type_id = self.token_type_ids
        label = self.labels[idx]

        return {
            "input_ids": input_id,
            "bboxes":bbox,
            "attention_mask": attention_mask,
            "token_type_ids":token_type_id,
            "ner_tags": label
        }

In [47]:
LayoutLMDataset = LayoutLMDataset(data_result["input_ids"],data_result["bboxes"],data_result["attention_mask"],data_result["token_type_ids"],data_result["ner_tags"])
LayoutLMDataset

In [48]:
dataloader = DataLoader(LayoutLMDataset,batch_size=1, shuffle=True)

In [51]:
batch = next(iter(dataloader))
inputs= batch
inputs

{'input_ids': tensor([0]),
 'bboxes': [tensor([0]), tensor([0]), tensor([0]), tensor([0])],
 'attention_mask': tensor([0]),
 'token_type_ids': [tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  tensor([0]),
  t

### 3.4 Prepare trainer for model - TRAINER

#4 Training procedure

In [ ]:
optimizer = AdamW(model.parameters(), lr=4e-5)
num_epochs = 5


for epoch in range(num_epochs):
    print("Epoch:", epoch)
    training_loss = 0.0
    model.train()
    for batch in tqdm(train_dataloader):
        labels = batch["ner_tags"].to(device)
        outputs = model(
            input_ids=batch["input_ids"].to(device), bbox=batch["bboxes"].to(device),
            attention_mask=batch["attention_mask"].to(device),
            token_type_ids=batch["token_type_ids"].to(device),
            labels=labels
        )
        loss = outputs.loss

        training_loss += loss.item()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    print("Training Loss:", training_loss / batch["input_ids"].shape[0])

    validation_loss = 0.0
    for batch in tqdm(valid_dataloader):
        labels = batch["ner_tags"].to(device)
        outputs = model(
            input_ids=batch["input_ids"].to(device), bbox=batch["bboxes"].to(device),
            attention_mask=batch["attention_mask"].to(device),
            token_type_ids=batch["token_type_ids"].to(device),
            labels=labels
        )
        loss = outputs.loss

        validation_loss += loss.item()

    print("Validation Loss:", validation_loss / batch["input_ids"].shape[0])

In [ ]:
model.save_pretrained('saved_model/')